In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sabbir5622r/july-revolution-sentiment-analysis-dataset-bangla")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/datasets/sabbir5622r/july-revolution-sentiment-analysis-dataset-bangla


In [5]:
import pandas as pd

data_file = "/kaggle/input/datasets/sabbir5622r/july-revolution-sentiment-analysis-dataset-bangla/student_people_mass_uprising_public_sentiments_dataset/student_people_mass_uprising_public_sentiments_dataset.csv"

df = pd.read_csv(data_file)

# Quick check
print(df.head())
print(df['label'].value_counts())

                                             comment language     platform  \
0  কোন সংলাপ হবে না। পদত্যাগ চাই এবং কঠোর শাস্ত...   Bangla     Facebook   
1  সব কয়টা রাজনৈতিক করে,এরা কেউ সাধারন শিক্ষার্থী নয়   Bangla      Youtube   
2  অপরাধের কোন ধর্ম দলীয় পরিচয় নেই। অপরাধী রাষ্...   Bangla     Facebook   
3               ইউনুস সরকার হিন্দুদের অত্যাচার করছে।   Bangla     Facebook   
4        ছাত্ররা দেশের ভবিষ্যৎ নির্মাণের মূল কারিগর।   Bangla  Twitter (X)   

      label  
0  Positive  
1  Negative  
2   Neutral  
3  Negative  
4  Positive  
label
Positive    1400
Negative    1400
Neutral     1400
Name: count, dtype: int64


In [6]:
# Map labels directly (dataset e label already string)
label_map = {"Positive": "Positive", "Negative": "Negative", "Neutral": "Neutral"}

# Instruction-response format, correct column name: 'comment'
df['text_input'] = "Classify sentiment: " + df['comment'].astype(str)
df['text_output'] = df['label'].map(label_map)

# Keep only necessary columns
df = df[['text_input', 'text_output']]
print(df.head())

                                          text_input text_output
0  Classify sentiment: কোন সংলাপ হবে না। পদত্যাগ...    Positive
1  Classify sentiment: সব কয়টা রাজনৈতিক করে,এরা ক...    Negative
2  Classify sentiment: অপরাধের কোন ধর্ম দলীয় পরি...     Neutral
3  Classify sentiment: ইউনুস সরকার হিন্দুদের অত্য...    Negative
4  Classify sentiment: ছাত্ররা দেশের ভবিষ্যৎ নির্...    Positive


In [11]:
from transformers import AutoTokenizer
from datasets import Dataset

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

# Create HuggingFace Dataset from pandas DataFrame
dataset = Dataset.from_pandas(df)

# Batched=True hole batch e list ashe, tai list comprehension use korte hobe
def tokenize(batch):
    texts = [inp + " " + out for inp, out in zip(batch["text_input"], batch["text_output"])]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=16)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize, batched=True)

# Check the first tokenized example
print(tokenized_dataset[0])

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

{'text_input': 'Classify sentiment: কোন সংলাপ হবে না। পদত্যাগ চাই এবং কঠোর শাস্তি চাই', 'text_output': 'Positive', 'input_ids': [2, 212107, 25627, 235292, 24313, 236180, 35243, 119877, 60648, 236955, 44663, 79618, 136624, 235940, 28596, 237273], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
!pip install -q --upgrade transformers
!pip install -q --upgrade peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 9.4 MB/s eta 0:00:00a 0:00:01


In [1]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

2026-02-14 08:47:07.202699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771058827.428785      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771058827.491685      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771058828.032717      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771058828.032752      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771058828.032754      55 computation_placer.cc:177] computation placer alr

In [3]:
# 0️⃣ Install required packages (run once in Kaggle)
!pip install -U torch transformers peft accelerate trl datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 109.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 87.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
#MAIN CODE FULL MODEL FINTETUNED

import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
)
from huggingface_hub import login
import numpy as np
import shutil
import gc

# ============================================================================
# STEP 1: Setup and Authentication
# ============================================================================
print("🔐 Logging into Hugging Face...")
login("hf_token")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📱 Using device: {device}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ============================================================================
# DISK SPACE CHECK
# ============================================================================
def check_disk_space():
    """Check available disk space"""
    stat = shutil.disk_usage("/")
    free_gb = stat.free / (1024**3)
    print(f"💾 Free disk space: {free_gb:.2f} GB")
    if free_gb < 5:
        print("⚠️  WARNING: Low disk space! May cause save errors.")
    return free_gb

check_disk_space()

# Clean up old checkpoints if they exist
if os.path.exists("./gemma2b_sentiment"):
    print("🧹 Cleaning old checkpoints...")
    shutil.rmtree("./gemma2b_sentiment", ignore_errors=True)

# ============================================================================
# STEP 2: Load Dataset
# ============================================================================
print("\n📊 Loading dataset...")
data_file = "/kaggle/input/datasets/sabbir5622r/july-revolution-sentiment-analysis-dataset-bangla/student_people_mass_uprising_public_sentiments_dataset/student_people_mass_uprising_public_sentiments_dataset.csv"

df = pd.read_csv(data_file)
print(f"Dataset shape: {df.shape}")
print(f"Label distribution:\n{df['label'].value_counts()}")

# ============================================================================
# STEP 3: Format Data
# ============================================================================
print("\n📝 Formatting data...")

def format_prompt(row):
    """Simple, consistent format"""
    text = str(row['comment']).strip()
    label = str(row['label']).strip()
    
    instruction = f"Sentiment: {text}\nLabel:"
    response = f" {label}"
    
    return instruction, response

df['instruction'] = df.apply(lambda x: format_prompt(x)[0], axis=1)
df['response'] = df.apply(lambda x: format_prompt(x)[1], axis=1)

# Proper split with stratification
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(
    df[['instruction', 'response']], 
    test_size=0.15, 
    random_state=42,
    stratify=df['label']
)

print(f"Train samples: {len(train_df)}, Eval samples: {len(eval_df)}")

# ============================================================================
# STEP 4: Load Tokenizer and Model
# ============================================================================
print("\n🤖 Loading tokenizer and model...")
model_name = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"

print(f"✓ Vocab size: {len(tokenizer)}")
print(f"✓ Pad token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")

use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
print(f"✓ Using BF16: {use_bf16}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if use_bf16 else torch.float32,
    device_map="auto",
)

model.config.use_cache = False
model.gradient_checkpointing_enable()

print(f"✓ Model loaded on: {model.device}")

# ============================================================================
# STEP 5: Tokenization with Proper Label Masking
# ============================================================================
print("\n🔤 Tokenizing...")

def tokenize_function(examples):
    """Tokenize with proper label masking"""
    model_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for instruction, response in zip(examples['instruction'], examples['response']):
        full_text = instruction + response
        
        inst_tokens = tokenizer(
            instruction,
            add_special_tokens=True,
            truncation=True,
            max_length=110,
        )
        
        full_tokens = tokenizer(
            full_text,
            add_special_tokens=True,
            truncation=True,
            max_length=128,
        )
        
        inst_len = len(inst_tokens["input_ids"])
        full_len = len(full_tokens["input_ids"])
        
        labels = [-100] * inst_len + full_tokens["input_ids"][inst_len:]
        
        # Ensure at least 1 non-masked token
        non_masked = sum(1 for l in labels if l != -100)
        if non_masked == 0:
            labels[-1] = full_tokens["input_ids"][-1]
        
        labels = labels[:full_len]
        
        model_inputs["input_ids"].append(full_tokens["input_ids"])
        model_inputs["attention_mask"].append(full_tokens["attention_mask"])
        model_inputs["labels"].append(labels)
    
    return model_inputs

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True))

print("Tokenizing datasets...")
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=50,
    remove_columns=train_dataset.column_names,
)

eval_dataset = eval_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=50,
    remove_columns=eval_dataset.column_names,
)

print(f"✓ Train: {len(train_dataset)}, Eval: {len(eval_dataset)}")

# ============================================================================
# STEP 6: Data Collator
# ============================================================================
from dataclasses import dataclass
from typing import Any, Dict, List

@dataclass
class DataCollatorForSentimentLM:
    """Custom collator with proper padding"""
    tokenizer: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        max_length = max(len(f["input_ids"]) for f in features)
        
        batch = {
            "input_ids": [],
            "attention_mask": [],
            "labels": []
        }
        
        for feature in features:
            input_ids = feature["input_ids"]
            attention_mask = feature["attention_mask"]
            labels = feature["labels"]
            
            padding_length = max_length - len(input_ids)
            
            input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
            attention_mask = attention_mask + [0] * padding_length
            labels = labels + [-100] * padding_length
            
            batch["input_ids"].append(input_ids)
            batch["attention_mask"].append(attention_mask)
            batch["labels"].append(labels)
        
        batch = {
            k: torch.tensor(v, dtype=torch.long) 
            for k, v in batch.items()
        }
        
        return batch

data_collator = DataCollatorForSentimentLM(tokenizer=tokenizer)

# ============================================================================
# STEP 7: OPTIMIZED Training Arguments (DISK SPACE FIX)
# ============================================================================
print("\n⚙️  Setting up training (disk-optimized)...")

training_args = TrainingArguments(
    output_dir="./gemma2b_sentiment",
    
    # Training
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    
    # Learning rate
    learning_rate=2e-5,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    
    # Optimization
    bf16=use_bf16,
    fp16=False,
    optim="adamw_torch",
    max_grad_norm=1.0,
    
    # Logging
    logging_dir="./logs",
    logging_steps=25,
    
    # ⭐ CRITICAL: Reduced saving to prevent disk errors
    eval_strategy="steps",
    eval_steps=200,  # Less frequent (was 100)
    save_strategy="steps",
    save_steps=200,  # Less frequent (was 100)
    save_total_limit=1,  # Keep ONLY 1 checkpoint (was 2)
    
    # ⭐ CRITICAL: Disable optimizer state saving (saves ~2GB per checkpoint)
    save_only_model=True,  # Don't save optimizer/scheduler states
    
    # Other
    report_to="none",
    load_best_model_at_end=False,  # Disabled to save space
    
    # Memory
    dataloader_num_workers=0,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    
    # ⭐ Disable safetensors temporarily if causing issues
    save_safetensors=False,  # Use regular pytorch format
)

print(f"✓ Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"✓ Checkpoints: Every {training_args.save_steps} steps, keeping max {training_args.save_total_limit}")

# ============================================================================
# STEP 8: Initialize Trainer
# ============================================================================
print("\n🎯 Initializing Trainer...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("✓ Trainer ready!")

# ============================================================================
# STEP 9: Sanity Check
# ============================================================================
print("\n🔍 Running sanity check...")

try:
    sample_batch = data_collator([train_dataset[0], train_dataset[1]])
    sample_batch = {k: v.to(model.device) for k, v in sample_batch.items()}
    
    with torch.no_grad():
        outputs = model(**sample_batch)
        loss = outputs.loss.item()
        print(f"✓ Test loss: {loss:.4f} (valid: {not torch.isnan(outputs.loss)})")
    
    del sample_batch, outputs
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✅ Sanity check passed!")

except Exception as e:
    print(f"❌ Sanity check failed: {e}")
    exit(1)

# ============================================================================
# STEP 10: Training with Cleanup
# ============================================================================
print("\n" + "="*70)
print("🚀 STARTING TRAINING...")
print("="*70 + "\n")

try:
    # Check space before training
    free_space = check_disk_space()
    
    if free_space < 3:
        print("⚠️  WARNING: Very low disk space. Training may fail.")
        print("Consider reducing save_steps or disabling checkpointing entirely.")
    
    train_result = trainer.train()
    
    print("\n" + "="*70)
    print("✅ TRAINING COMPLETED!")
    print("="*70)
    
    print("\n📊 Training Metrics:")
    for key, value in train_result.metrics.items():
        print(f"  {key}: {value}")
    
    # Save ONLY the final model
    print("\n💾 Saving final model...")
    final_dir = "./gemma2b_sentiment_final"
    
    # Clean up intermediate checkpoints first
    print("🧹 Cleaning intermediate checkpoints...")
    for item in os.listdir("./gemma2b_sentiment"):
        if item.startswith("checkpoint-"):
            shutil.rmtree(f"./gemma2b_sentiment/{item}", ignore_errors=True)
    
    trainer.save_model(final_dir)
    tokenizer.save_pretrained(final_dir)
    print(f"✓ Saved to: {final_dir}")
    
    # Final eval
    print("\n📊 Final evaluation...")
    eval_results = trainer.evaluate()
    print("\nFinal Metrics:")
    for key, value in eval_results.items():
        if not np.isnan(value):
            print(f"  {key}: {value:.4f}")

except Exception as e:
    print(f"\n⚠️  Training encountered error: {e}")
    print("\n💡 Model may still be usable if training progressed far enough.")
    import traceback
    traceback.print_exc()

# Cleanup
torch.cuda.empty_cache()
gc.collect()

# ============================================================================
# STEP 11: Test Predictions
# ============================================================================
print("\n" + "="*70)
print("🧪 TESTING PREDICTIONS")
print("="*70 + "\n")

model.eval()

def predict(text):
    """Predict sentiment"""
    prompt = f"Sentiment: {text}\nLabel:"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=110).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.01,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    label = result.split("Label:")[-1].strip()
    
    del inputs, outputs
    torch.cuda.empty_cache()
    
    return label

# Test samples
tests = [
    ("ছাত্ররা দেশের ভবিষ্যৎ", "Positive"),
    ("সরকার অত্যাচার করছে", "Negative"),
    ("অপরাধের কোন ধর্ম নেই", "Neutral"),
    ("জুলাই বিপ্লব ছিল একটি ঐতিহাসিক ঘটনা", "Positive"),
    ("দুর্নীতি বন্ধ করতে হবে", "Negative"),
]

print("Testing predictions:\n")
correct = 0
for text, expected in tests:
    pred = predict(text)
    match = expected.lower() in pred.lower()
    correct += int(match)
    status = '✓' if match else '✗'
    print(f"{status} Text: {text[:50]}...")
    print(f"  Expected: {expected}, Got: {pred}\n")

print(f"Accuracy: {correct}/{len(tests)} = {100*correct/len(tests):.1f}%")

print("\n" + "="*70)
print("🎉 DONE!")
print("="*70)

# Final disk check
check_disk_space()

2026-02-14 11:18:56.585508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771067936.792099      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771067936.854609      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771067937.399100      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771067937.399136      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771067937.399138      55 computation_placer.cc:177] computation placer alr

🔐 Logging into Hugging Face...
📱 Using device: cuda
✓ GPU Memory: 15.64 GB
💾 Free disk space: 1426.69 GB

📊 Loading dataset...
Dataset shape: (4200, 4)
Label distribution:
label
Positive    1400
Negative    1400
Neutral     1400
Name: count, dtype: int64

📝 Formatting data...
Train samples: 3570, Eval samples: 630

🤖 Loading tokenizer and model...


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...


✅ TRAINING COMPLETED!

📊 Training Metrics:
  train_runtime: 2411.4058
  train_samples_per_second: 1.48
  train_steps_per_second: 0.185
  total_flos: 2350792381194240.0
  train_loss: 0.7927555259175482
  epoch: 1.0

💾 Saving final model...
🧹 Cleaning intermediate checkpoints...
✓ Saved to: ./gemma2b_sentiment_final

📊 Final evaluation...



Final Metrics:
  eval_loss: 0.4213
  eval_runtime: 95.5946
  eval_samples_per_second: 6.5900
  eval_steps_per_second: 3.2950
  epoch: 1.0000


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🧪 TESTING PREDICTIONS

Testing predictions:

✓ Text: ছাত্ররা দেশের ভবিষ্যৎ...
  Expected: Positive, Got: Positive

✓ Text: সরকার অত্যাচার করছে...
  Expected: Negative, Got: Negative

✓ Text: অপরাধের কোন ধর্ম নেই...
  Expected: Neutral, Got: Neutral

✓ Text: জুলাই বিপ্লব ছিল একটি ঐতিহাসিক ঘটনা...
  Expected: Positive, Got: Positive

✗ Text: দুর্নীতি বন্ধ করতে হবে...
  Expected: Negative, Got: Positive

Accuracy: 4/5 = 80.0%

🎉 DONE!
💾 Free disk space: 1422.00 GB


1422.0008583068848

In [2]:
"""
Gradio Interface for Finetuned Gemma-2B Bangla Sentiment Analysis
Simple and beautiful UI for testing the model
"""

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

# ============================================================================
# Load Model and Tokenizer
# ============================================================================
print("🔄 Loading finetuned model...")

MODEL_PATH = "./gemma2b_sentiment_final"  # Change if your path is different

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
    # Setup padding
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    # Load model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16 if use_bf16 else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
    )
    model.eval()
    
    print(f"✅ Model loaded successfully on {device}")
    print(f"✅ Using BF16: {use_bf16}")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\n💡 Make sure the model path is correct and model is saved.")
    raise

# ============================================================================
# Prediction Function
# ============================================================================
def predict_sentiment(text, show_confidence=False):
    """
    Predict sentiment from Bangla text
    
    Args:
        text: Input Bangla text
        show_confidence: Whether to show confidence scores (experimental)
    
    Returns:
        Sentiment label and optional confidence
    """
    if not text or text.strip() == "":
        return "⚠️ দয়া করে কিছু লিখুন (Please enter some text)", ""
    
    try:
        # Format prompt
        prompt = f"Sentiment: {text.strip()}\nLabel:"
        
        # Tokenize
        inputs = tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=110
        ).to(model.device)
        
        # Generate
        start_time = time.time()
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                temperature=0.01,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                return_dict_in_generate=True,
                output_scores=show_confidence,
            )
        
        inference_time = time.time() - start_time
        
        # Decode
        result = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        label = result.split("Label:")[-1].strip()
        
        # Clean up label
        label = label.split()[0] if label else "Unknown"
        
        # Map to Bangla + English
        label_map = {
            "Positive": "😊 Positive (ইতিবাচক)",
            "Negative": "😞 Negative (নেতিবাচক)",
            "Neutral": "😐 Neutral (নিরপেক্ষ)",
        }
        
        display_label = label_map.get(label, f"❓ {label}")
        
        # Create info text
        info = f"⏱️ Inference time: {inference_time:.3f}s"
        
        # Optional: Add confidence (experimental)
        if show_confidence and hasattr(outputs, 'scores') and len(outputs.scores) > 0:
            try:
                # Get probabilities for first generated token
                first_token_scores = outputs.scores[0][0]
                probs = torch.nn.functional.softmax(first_token_scores, dim=-1)
                confidence = probs.max().item() * 100
                info += f"\n🎯 Confidence: {confidence:.1f}%"
            except:
                pass
        
        # Cleanup
        del inputs, outputs
        torch.cuda.empty_cache()
        
        return display_label, info
        
    except Exception as e:
        return f"❌ Error: {str(e)}", ""

# ============================================================================
# Example Texts
# ============================================================================
examples = [
    ["ছাত্ররা দেশের ভবিষ্যৎ"],
    ["সরকার জনগণের উপর অত্যাচার করছে"],
    ["অপরাধের কোন ধর্ম নেই"],
    ["জুলাই বিপ্লব ছিল একটি ঐতিহাসিক ঘটনা"],
    ["দুর্নীতি বন্ধ করতে হবে"],
    ["শিক্ষার্থীরা আন্দোলনে অংশ নিচ্ছে"],
    ["আমরা ন্যায়বিচার চাই"],
    ["পুলিশ নিরপরাধ মানুষকে মারছে"],
    ["দেশে শান্তি প্রতিষ্ঠিত হয়েছে"],
    ["এটি একটি সাধারণ ঘটনা"],
]

# ============================================================================
# Gradio Interface
# ============================================================================
def create_interface():
    """Create beautiful Gradio interface"""
    
    with gr.Blocks(
        theme=gr.themes.Soft(),
        title="Bangla Sentiment Analysis - July Revolution"
    ) as demo:
        
        # Header
        gr.Markdown(
            """
            # 🇧🇩 Bangla Sentiment Analysis
            ### July Revolution Dataset - Finetuned Gemma-2B
            
            এই মডেলটি বাংলা টেক্সট থেকে Sentiment (ইতিবাচক/নেতিবাচক/নিরপেক্ষ) শনাক্ত করে।  
            This model detects sentiment (Positive/Negative/Neutral) from Bangla text.
            """
        )
        
        with gr.Row():
            with gr.Column(scale=2):
                # Input
                text_input = gr.Textbox(
                    label="📝 বাংলা টেক্সট লিখুন (Enter Bangla Text)",
                    placeholder="এখানে আপনার বাংলা টেক্সট লিখুন...",
                    lines=4,
                )
                
                # Options
                with gr.Row():
                    confidence_check = gr.Checkbox(
                        label="Show confidence (experimental)",
                        value=False,
                    )
                
                # Buttons
                with gr.Row():
                    submit_btn = gr.Button("🔍 Analyze Sentiment", variant="primary", size="lg")
                    clear_btn = gr.ClearButton([text_input], value="🗑️ Clear")
            
            with gr.Column(scale=1):
                # Output
                sentiment_output = gr.Textbox(
                    label="📊 Sentiment Result",
                    interactive=False,
                    lines=2,
                )
                
                info_output = gr.Textbox(
                    label="ℹ️ Info",
                    interactive=False,
                    lines=2,
                )
        
        # Examples
        gr.Markdown("### 📋 Example Texts (নমুনা টেক্সট)")
        gr.Examples(
            examples=examples,
            inputs=text_input,
            label=None,
        )
        
        # Footer
        gr.Markdown(
            """
            ---
            **Model:** Google Gemma-2B (Finetuned)  
            **Dataset:** July Revolution Sentiment Analysis (4200 samples)  
            **Labels:** Positive (ইতিবাচক) | Negative (নেতিবাচক) | Neutral (নিরপেক্ষ)
            """
        )
        
        # Event handlers
        submit_btn.click(
            fn=predict_sentiment,
            inputs=[text_input, confidence_check],
            outputs=[sentiment_output, info_output],
        )
        
        text_input.submit(
            fn=predict_sentiment,
            inputs=[text_input, confidence_check],
            outputs=[sentiment_output, info_output],
        )
    
    return demo

# ============================================================================
# Launch
# ============================================================================
if __name__ == "__main__":
    print("\n" + "="*70)
    print("🚀 Launching Gradio Interface...")
    print("="*70 + "\n")
    
    demo = create_interface()
    
    # Launch with options
    demo.launch(
        share=True,  # Create public link
        server_name="0.0.0.0",  # Allow external access
        server_port=7860,  # Default Gradio port
        show_error=True,
        # Uncomment below for authentication
        # auth=("admin", "password123"),
    )

🔄 Loading finetuned model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully on cuda
✅ Using BF16: True

🚀 Launching Gradio Interface...

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://d48ddb14f0c4a387c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
